In [11]:
import tensorflow as tf
import numpy as np
import pandas as pd


read_csv  = pd.read_csv('dataset.csv')
oracle_statements = read_csv['Source_Statement'].apply(str).values.tolist()
postgres_statements = read_csv['Target_Statement'].apply(str).values.tolist()
# # Define the training data
# postgres_statements = [
#     "SELECT * FROM employees",
#     "INSERT INTO customers (name, email) VALUES ('John Doe', 'john@example.com')",
#     "UPDATE products SET price = 9.99 WHERE id = 123",
#     "DELETE FROM orders WHERE status = 'cancelled'",
# ]

# oracle_statements = [
#     "SELECT * FROM employees",
#     "INSERT INTO customers (name, email) VALUES ('John Doe', 'john@example.com')",
#     "UPDATE products SET price = 9.99 WHERE id = 123",
#     "DELETE FROM orders WHERE status = 'cancelled'",
# ]

# Create a tokenizer
tokenizer = tf.keras.preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(postgres_statements + oracle_statements)

# Convert text to sequences
postgres_sequences = tokenizer.texts_to_sequences(postgres_statements)
oracle_sequences = tokenizer.texts_to_sequences(oracle_statements)

# Pad sequences
max_length = max(max(len(seq) for seq in postgres_sequences), max(len(seq) for seq in oracle_sequences))
postgres_sequences = tf.keras.preprocessing.sequence.pad_sequences(postgres_sequences, maxlen=max_length)
oracle_sequences = tf.keras.preprocessing.sequence.pad_sequences(oracle_sequences, maxlen=max_length)

# Define the model
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(len(tokenizer.word_index) + 1, 128, input_length=max_length),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(oracle_statements), activation='softmax')
])

# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Convert oracle statements to one-hot encoded vectors
oracle_labels = tf.keras.utils.to_categorical(np.arange(len(oracle_statements)), num_classes=len(oracle_statements))

# Train the model
model.fit(postgres_sequences, oracle_labels, epochs=100, batch_size=16)



Epoch 1/100
3/3 [==============================] - 5s 155ms/step - loss: 3.5901 - accuracy: 0.0556
Epoch 2/100
3/3 [==============================] - 0s 153ms/step - loss: 3.5783 - accuracy: 0.0278
Epoch 3/100
3/3 [==============================] - 0s 151ms/step - loss: 3.5685 - accuracy: 0.1111
Epoch 4/100
3/3 [==============================] - 0s 163ms/step - loss: 3.5605 - accuracy: 0.1111
Epoch 5/100
3/3 [==============================] - 0s 163ms/step - loss: 3.5499 - accuracy: 0.1389
Epoch 6/100
3/3 [==============================] - 0s 158ms/step - loss: 3.5357 - accuracy: 0.1111
Epoch 7/100
3/3 [==============================] - 0s 158ms/step - loss: 3.5179 - accuracy: 0.0833
Epoch 8/100
3/3 [==============================] - 1s 172ms/step - loss: 3.4889 - accuracy: 0.0833
Epoch 9/100
3/3 [==============================] - 0s 161ms/step - loss: 3.4554 - accuracy: 0.1111
Epoch 10/100
3/3 [==============================] - 1s 163ms/step - loss: 3.4020 - accuracy: 0.0833
Epoch 11/

In [10]:
# Test the model
test_statement = """open op_cursor for select distinct ca.leveldetailname as location, awf.billno as billnumber, pb.billdate as billdate, coalesce( c.customername, 'NA' )
as payer, bc.registrationno as uhid, pb.patientidentifiernumber as ipnumber, pb.patientname, round( pb.totalbillamount )
as billamount, awf.oldbillingamount as cancelledamount, awf.completedon as dateofcancellation, um.firstname || ' ' || um.middlename || ' ' || um.lastname as cancelledby, coalesce( lov.lovdetaildescription, lov.lovdetailvalue )
as reasoncode, bc.remarks, nb.billno as newbillnumber, nb.billdate as dateofnewbill,( select listagg( um.firstname || ' ' || um.middlename || ' ' || um.lastname, ',' )within group( order by bapwf.billingapwftransid asc )from billing.ageingvalueapprovalwftrans bapwf inner join billing.ageingvalueapprovalworkflow awf on awf.billingapprovalwfid = bapwf.billingapprovalwfid inner join billing.usermaster um on um.employeeid = bapwf.wipby where awf.billno = bc.billno and awf.flowname = 'Billing Cancellation' )
as approvedby, awf.workflowid from billing.ageingvalueapprovalworkflow awf inner join billing.ageingvalueapprovalwftrans bapwf on awf.billingapprovalwfid = bapwf.billingapprovalwfid inner join billing.billcancellation bc on bc.billno = awf.billno inner join billing.lovdetail lov on lov.lovdetailid = bc.cancelreasonid inner join billing.patientbill pb on pb.billno = bc.billno left outer join billing.gtt_billcancellednewbill nb on pb.patientidentifiernumber = 'CAN' || nb.ipno left outer join crm.customers c on c.customerid = pb.primarypayerid inner join billing.usermaster um on um.employeeid = bapwf.createdby inner join ehis.coa_struct_details ca on ca.chartid = awf.locationid where( date_trunc( 'day', awf.createddate )between iv_fromdate and iv_todate )and( iv_locationid is null or pb.locationid = iv_locationid )and( iv_payertypeid is null or c.customerid = iv_payertypeid )and( iv_intiatedby is null or awf.createdby = iv_intiatedby )and( iv_approvedby is null or bapwf.wipby = iv_approvedby )and awf.flowname = 'Billing Cancellation' and awf.status = 1 order by dateofcancellation asc ;"""
test_sequence = tokenizer.texts_to_sequences([test_statement])
test_sequence = tf.keras.preprocessing.sequence.pad_sequences(test_sequence, maxlen=max_length)
predictions = model.predict(test_sequence)
predicted_index = np.argmax(predictions)
predicted_statement = oracle_statements[predicted_index]

print(f"Predicted Oracle statement: {predicted_statement}")


1/1 [==============================] - 0s 38ms/step
Predicted Oracle statement: OPEN OP_CURSOR FOR
 SELECT DISTINCT CA.LEVELDETAILNAME as Location,
 AWF.Billno as BillNumber,
 PB.Billdate as Billdate,
 NVL(C.Customername, 'NA') as Payer,
 BC.REGISTRATIONNO as UHID,
 PB.Patientidentifiernumber as IPNUMBER,
 PB.Patientname,
 ROUND(PB.Totalbillamount) as BillAmount,
 AWF.OLDBILLINGAMOUNT as CancelledAmount,
 AWF.COMPLETEDON as DateofCancellation,
 UM.FIRSTNAME || ' ' || UM.MIDDLENAME || ' ' ||
 UM.LASTNAME as Cancelledby,
 NVL(Lov.Lovdetaildescription, Lov.Lovdetailvalue) as Reasoncode,
 BC.Remarks,
 NB.BILLNO as NEWBILLNUMBER,
 NB.BILLDATE as DateofNewbill,
 (SELECT LISTAGG(UM.FIRSTNAME || ' ' || UM.MIDDLENAME || ' ' ||
 UM.LASTNAME,
 ',') WITHIN
 GROUP(
 order by BAPWF.Billingapwftransid asc)
 from BILLING.AGEINGVALUEAPPROVALWFTRANS bapwf
 inner join BILLING.AGEINGVALUEAPPROVALWORKFLOW AWF on AWF.BILLINGAPPROVALWFID =
 BAPWF.BILLINGAPPROVALWFID
 inner join billing.usermaster um on um.EM